In [2]:
import pandas as pd
import os
import re

# フォルダと出力先
shap_dir = "results"  # SHAP影響度CSVファイルがあるフォルダ
output_dir = "results_summary"
os.makedirs(output_dir, exist_ok=True)

# 対象の比率
ratios = ["1_to_2", "1_to_5", "1_to_10", "1_to_20"]

# SHAPファイルパターン：例）Combined_CatBoost_SHAP影響度_1_to_2.csv
pattern = re.compile(r"(.+?)_(.+?)_SHAP影響度_([0-9a-zA-Z_]+)\.csv")

# ファイル統合用リスト
all_dfs = []

# SHAPファイル読み込みループ
for file in os.listdir(shap_dir):
    if file.endswith(".csv"):
        match = pattern.match(file)
        if match:
            structure, model, ratio = match.groups()
            if ratio in ratios:
                df = pd.read_csv(os.path.join(shap_dir, file))
                df["構成"] = structure
                df["モデル"] = model
                df["データ比"] = ratio
                all_dfs.append(df)

# DataFrame統合
if not all_dfs:
    raise RuntimeError("対象のSHAPファイルが見つかりませんでした。")
combined_df = pd.concat(all_dfs, ignore_index=True)

# SHAP影響度の上位10特徴量を集計（全構成・全モデル・全データ比横断）
top10 = (
    combined_df.groupby("Feature")["SHAP平均影響度"]
    .mean()
    .reset_index()
    .sort_values("SHAP平均影響度", ascending=False)
    .head(10)
)

# 出力
output_file = os.path.join(output_dir, "SHAP影響度_トップ10特徴量_統合.csv")
top10.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"上位10特徴量を {output_file} に保存しました。")
print(top10)


上位10特徴量を results_summary\SHAP影響度_トップ10特徴量_統合.csv に保存しました。
                Feature     SHAP平均影響度
7505    損益計算書_不動産事業等売上高  45533.032826
7503   損益計算書_不動産事業等売上原価  45425.195524
7502     損益計算書_不動産事業売上高  19491.849204
7500    損益計算書_不動産事業売上原価  18865.089158
6227           原価明細書_小計   7031.969615
9893         損益計算書_給料手当   4327.385919
6485    原価明細書_期末原材料たな卸高   3985.953543
7049      損益計算書_その他事業原価   3115.545925
10717     損益計算書_関係会社清算損   3009.871914
6864      原価明細書_（うち人件費）   2621.990630


In [3]:
import pandas as pd

# 前提：combined_df には '構成', 'モデル', 'データ比', 'Feature', 'SHAP平均影響度' が含まれている

# 結果格納リスト
grouped_top10 = []

# 各構成×モデル×データ比の組み合わせでループ
for (structure, model, ratio), group in combined_df.groupby(["構成", "モデル", "データ比"]):
    top10 = (
        group.groupby("Feature")["SHAP平均影響度"]
        .mean()
        .reset_index()
        .sort_values("SHAP平均影響度", ascending=False)
        .head(10)
    )
    top10["構成"] = structure
    top10["モデル"] = model
    top10["データ比"] = ratio
    grouped_top10.append(top10)

# 全体を1つのDataFrameにまとめる
top10_all = pd.concat(grouped_top10, ignore_index=True)

# 列の並びを調整
top10_all = top10_all[["構成", "モデル", "データ比", "Feature", "SHAP平均影響度"]]

# 保存
output_path = os.path.join(output_dir, "SHAP影響度_トップ10特徴量_全構成モデル比率別.csv")
top10_all.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"構成・モデル・データ比別のSHAP影響度トップ10特徴量を {output_path} に保存しました。")


構成・モデル・データ比別のSHAP影響度トップ10特徴量を results_summary\SHAP影響度_トップ10特徴量_全構成モデル比率別.csv に保存しました。


In [4]:
import pandas as pd

# 結果格納リスト
normalized_all = []

# 各構成×モデル×データ比でループ
for (structure, model, ratio), group in combined_df.groupby(["構成", "モデル", "データ比"]):
    shap_df = (
        group.groupby("Feature")["SHAP平均影響度"]
        .mean()
        .reset_index()
    )

    # 正規化（合計を1に）
    shap_df["正規化SHAP影響度"] = shap_df["SHAP平均影響度"] / shap_df["SHAP平均影響度"].sum()
    shap_df["構成"] = structure
    shap_df["モデル"] = model
    shap_df["データ比"] = ratio
    normalized_all.append(shap_df)

# 統合
normalized_df = pd.concat(normalized_all, ignore_index=True)

# 列の並びを調整
normalized_df = normalized_df[["構成", "モデル", "データ比", "Feature", "SHAP平均影響度", "正規化SHAP影響度"]]

# 保存
output_path = os.path.join(output_dir, "SHAP影響度_正規化_全特徴量.csv")
normalized_df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"構成・モデル・データ比別のSHAP影響度（正規化済み）を {output_path} に保存しました。")


構成・モデル・データ比別のSHAP影響度（正規化済み）を results_summary\SHAP影響度_正規化_全特徴量.csv に保存しました。


In [5]:
import pandas as pd
import os

# 正規化＆トップ10格納用リスト
normalized_all = []

# 各構成×モデル×データ比ごとに正規化処理とTop10抽出
for (structure, model, ratio), group in combined_df.groupby(["構成", "モデル", "データ比"]):
    shap_df = (
        group.groupby("Feature")["SHAP平均影響度"]
        .mean()
        .reset_index()
    )
    # 正規化
    shap_df["正規化SHAP影響度"] = shap_df["SHAP平均影響度"] / shap_df["SHAP平均影響度"].sum()

    # メタ情報付加
    shap_df["構成"] = structure
    shap_df["モデル"] = model
    shap_df["データ比"] = ratio

    # 上位10特徴量抽出
    top10 = shap_df.sort_values("正規化SHAP影響度", ascending=False).head(10)
    normalized_all.append(top10)

# 統合
normalized_df = pd.concat(normalized_all, ignore_index=True)

# 列の並びを整理
normalized_df = normalized_df[["データ比", "構成", "モデル", "Feature", "SHAP平均影響度", "正規化SHAP影響度"]]

# 保存フォルダとパス設定
output_dir = "results_summary"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "SHAP影響度_正規化_トップ10特徴量一覧.xlsx")

# Excel出力
normalized_df.to_excel(output_path, index=False)

print(f"正規化SHAP上位10特徴量を {output_path} に保存しました。")


正規化SHAP上位10特徴量を results_summary\SHAP影響度_正規化_トップ10特徴量一覧.xlsx に保存しました。


In [6]:
import pandas as pd
import xlsxwriter
import os

# 出力ファイルパス（必要に応じて変更）
output_path = "SHAP影響度_モデル別シート_色分け付き.xlsx"

# 正規化＆トップ10格納用リスト
normalized_all = []

# 各構成 × モデル × データ比 ごとに SHAP値正規化＋Top10抽出
for (structure, model, ratio), group in combined_df.groupby(["構成", "モデル", "データ比"]):
    shap_df = (
        group.groupby("Feature")["SHAP平均影響度"]
        .mean()
        .reset_index()
    )
    shap_df["正規化SHAP影響度"] = shap_df["SHAP平均影響度"] / shap_df["SHAP平均影響度"].sum()
    shap_df["構成"] = structure
    shap_df["モデル"] = model
    shap_df["データ比"] = ratio

    top10 = shap_df.sort_values("正規化SHAP影響度", ascending=False).head(10)
    normalized_all.append(top10)

# 結合して1つのデータフレームに
normalized_df = pd.concat(normalized_all, ignore_index=True)

# Excel出力準備
workbook = xlsxwriter.Workbook(output_path)

# モデルごとにシート作成
for model in normalized_df["モデル"].unique():
    worksheet = workbook.add_worksheet(model)
    model_df = normalized_df[normalized_df["モデル"] == model]
    model_df = model_df.sort_values(by=["データ比", "構成", "正規化SHAP影響度"], ascending=[True, True, False])

    # 色分け（Feature単位）
    feature_colors = {}
    color_palette = [
        "#D9E1F2", "#FCE4D6", "#E2EFDA", "#FFF2CC", "#F8CBAD",
        "#E4DFEC", "#DDEBF7", "#FBE4D5", "#DAE8FC", "#D5F5E3",
        "#F4CCCC", "#D9D2E9", "#C9DAF8", "#B6D7A8", "#FFE599"
    ]
    color_index = 0

    # ヘッダー
    headers = ["データ比", "構成", "Top", "Feature", "正規化SHAP影響度"]
    for col, header in enumerate(headers):
        worksheet.write(0, col, header)

    row = 1
    for (ratio, structure), group in model_df.groupby(["データ比", "構成"]):
        sorted_group = group.sort_values("正規化SHAP影響度", ascending=False).reset_index(drop=True)
        for rank, (_, row_data) in enumerate(sorted_group.iterrows(), start=1):
            feature = row_data["Feature"]
            # 新しい特徴量には新しい色を割り当て
            if feature not in feature_colors:
                feature_colors[feature] = workbook.add_format({"bg_color": color_palette[color_index % len(color_palette)]})
                color_index += 1
            fmt = feature_colors[feature]

            worksheet.write(row, 0, ratio)
            worksheet.write(row, 1, structure)
            worksheet.write(row, 2, rank)
            worksheet.write(row, 3, feature, fmt)
            worksheet.write(row, 4, row_data["正規化SHAP影響度"])
            row += 1

workbook.close()
print(f"✅ Excelファイルを出力しました: {output_path}")


✅ Excelファイルを出力しました: SHAP影響度_モデル別シート_色分け付き.xlsx


In [7]:
import pandas as pd
import xlsxwriter

# 正規化＆Top10抽出
normalized_all = []
for (structure, model, ratio), group in combined_df.groupby(["構成", "モデル", "データ比"]):
    shap_df = (
        group.groupby("Feature")["SHAP平均影響度"]
        .mean()
        .reset_index()
    )
    shap_df["正規化SHAP影響度"] = shap_df["SHAP平均影響度"] / shap_df["SHAP平均影響度"].sum()
    shap_df["構成"] = structure
    shap_df["モデル"] = model
    shap_df["データ比"] = ratio
    top10 = shap_df.sort_values("正規化SHAP影響度", ascending=False).head(10)
    top10["Top"] = range(1, 11)
    normalized_all.append(top10)

df = pd.concat(normalized_all, ignore_index=True)

# 出力先
output_path = "SHAP影響度_モデル別シート_横持ち.xlsx"
workbook = xlsxwriter.Workbook(output_path)

# モデルごとにシートを分けて処理
for model in df["モデル"].unique():
    model_df = df[df["モデル"] == model]

    # Pivot：構成・Topを行インデックス、データ比ごとに列展開（Feature名とSHAP値を横持ち）
    pivoted = model_df.pivot_table(
        index=["構成", "Top"],
        columns="データ比",
        values=["Feature", "正規化SHAP影響度"],
        aggfunc="first"
    )

    pivoted.columns = [f"{col[1]}_{col[0]}" for col in pivoted.columns]  # フラット化
    pivoted = pivoted.reset_index()

    # シート作成
    worksheet = workbook.add_worksheet(model)

    # カラー定義
    feature_colors = {}
    color_list = [
        "#D9E1F2", "#FCE4D6", "#E2EFDA", "#FFF2CC", "#F8CBAD",
        "#E4DFEC", "#DDEBF7", "#FBE4D5", "#DAE8FC", "#D5F5E3",
        "#F4CCCC", "#D9D2E9", "#C9DAF8", "#B6D7A8", "#FFE599"
    ]
    color_index = 0

    # ヘッダー出力
    for col_idx, col_name in enumerate(pivoted.columns):
        worksheet.write(0, col_idx, col_name)

    # データ出力（色付き）
    for row_idx, row in pivoted.iterrows():
        for col_idx, col_name in enumerate(pivoted.columns):
            value = row[col_name]
            if "Feature" in col_name and pd.notnull(value):
                if value not in feature_colors:
                    feature_colors[value] = workbook.add_format({"bg_color": color_list[color_index % len(color_list)]})
                    color_index += 1
                fmt = feature_colors[value]
                worksheet.write(row_idx + 1, col_idx, value, fmt)
            else:
                worksheet.write(row_idx + 1, col_idx, value)

workbook.close()
print(f"✅ モデルごとにシートを分けて、横持ち形式でSHAP上位10特徴量を出力しました: {output_path}")


✅ モデルごとにシートを分けて、横持ち形式でSHAP上位10特徴量を出力しました: SHAP影響度_モデル別シート_横持ち.xlsx


In [8]:
import pandas as pd
import xlsxwriter

# ★データ比の順番を明示的に定義
ratio_order = ["1_to_2", "1_to_5", "1_to_10", "1_to_20"]

# 正規化SHAP値をTop10で抽出
normalized_all = []
for (structure, model, ratio), group in combined_df.groupby(["構成", "モデル", "データ比"]):
    shap_df = (
        group.groupby("Feature")["SHAP平均影響度"]
        .mean()
        .reset_index()
    )
    shap_df["正規化SHAP影響度"] = shap_df["SHAP平均影響度"] / shap_df["SHAP平均影響度"].sum()
    shap_df["構成"] = structure
    shap_df["モデル"] = model
    shap_df["データ比"] = ratio
    top10 = shap_df.sort_values("正規化SHAP影響度", ascending=False).head(10)
    top10["Top"] = range(1, 11)
    normalized_all.append(top10)

df = pd.concat(normalized_all, ignore_index=True)

# Excel出力先
output_path = "SHAP影響度_モデル別シート_列順指定.xlsx"
workbook = xlsxwriter.Workbook(output_path)

# モデルごとに処理
for model in df["モデル"].unique():
    model_df = df[df["モデル"] == model]

    # Pivot処理（列展開）
    pivoted = model_df.pivot_table(
        index=["構成", "Top"],
        columns="データ比",
        values=["Feature", "正規化SHAP影響度"],
        aggfunc="first"
    )

    # 列順を ratio_order に基づいて再構築（Feature, SHAPを交互に）
    col_order = []
    for ratio in ratio_order:
        col_order.append(f"{ratio}_Feature")
        col_order.append(f"{ratio}_正規化SHAP影響度")

    # MultiIndexをフラットに変換
    pivoted.columns = [f"{col[1]}_{col[0]}" for col in pivoted.columns]
    pivoted = pivoted.reset_index()

    # 必要な列順で並べ替え（存在しない列は無視）
    existing_cols = ["構成", "Top"] + [col for col in col_order if col in pivoted.columns]
    pivoted = pivoted[existing_cols]

    # シート書き出し
    worksheet = workbook.add_worksheet(model)

    # 色指定：Feature名単位で共通色割り当て
    feature_colors = {}
    color_list = [
        "#D9E1F2", "#FCE4D6", "#E2EFDA", "#FFF2CC", "#F8CBAD",
        "#E4DFEC", "#DDEBF7", "#FBE4D5", "#DAE8FC", "#D5F5E3",
        "#F4CCCC", "#D9D2E9", "#C9DAF8", "#B6D7A8", "#FFE599"
    ]
    color_index = 0

    # ヘッダー出力
    for col_idx, col_name in enumerate(pivoted.columns):
        worksheet.write(0, col_idx, col_name)

    # データ出力（Feature列は色塗り）
    for row_idx, row in pivoted.iterrows():
        for col_idx, col_name in enumerate(pivoted.columns):
            value = row[col_name]
            if "Feature" in col_name and pd.notnull(value):
                if value not in feature_colors:
                    feature_colors[value] = workbook.add_format({"bg_color": color_list[color_index % len(color_list)]})
                    color_index += 1
                fmt = feature_colors[value]
                worksheet.write(row_idx + 1, col_idx, value, fmt)
            else:
                worksheet.write(row_idx + 1, col_idx, value)

workbook.close()
print(f"✅ 列順固定・モデル別シートのSHAP影響度ファイルを出力しました: {output_path}")


✅ 列順固定・モデル別シートのSHAP影響度ファイルを出力しました: SHAP影響度_モデル別シート_列順指定.xlsx


In [9]:
import pandas as pd
import xlsxwriter

# 並び順の明示的な指定
ratio_order = ["1_to_2", "1_to_5", "1_to_10", "1_to_20"]
structure_order = ["Standalone", "Consolidated", "Combined"]

# 正規化SHAP値とTop10の準備
normalized_all = []
for (structure, model, ratio), group in combined_df.groupby(["構成", "モデル", "データ比"]):
    shap_df = (
        group.groupby("Feature")["SHAP平均影響度"]
        .mean()
        .reset_index()
    )
    shap_df["正規化SHAP影響度"] = shap_df["SHAP平均影響度"] / shap_df["SHAP平均影響度"].sum()
    shap_df["構成"] = structure
    shap_df["モデル"] = model
    shap_df["データ比"] = ratio
    top10 = shap_df.sort_values("正規化SHAP影響度", ascending=False).head(10)
    top10["Top"] = range(1, 11)
    normalized_all.append(top10)

df = pd.concat(normalized_all, ignore_index=True)

# Excel出力設定
output_path = "SHAP影響度_モデル別シート_構成順付き.xlsx"
workbook = xlsxwriter.Workbook(output_path)

# モデルごとにシートを作成
for model in df["モデル"].unique():
    model_df = df[df["モデル"] == model]

    # Pivot（構成・Topで行、データ比で列展開）
    pivoted = model_df.pivot_table(
        index=["構成", "Top"],
        columns="データ比",
        values=["Feature", "正規化SHAP影響度"],
        aggfunc="first"
    )

    # 列フラット化
    pivoted.columns = [f"{col[1]}_{col[0]}" for col in pivoted.columns]
    pivoted = pivoted.reset_index()

    # 構成順に並び替え（カテゴリ型で明示）
    pivoted["構成"] = pd.Categorical(pivoted["構成"], categories=structure_order, ordered=True)
    pivoted = pivoted.sort_values(by=["構成", "Top"]).reset_index(drop=True)

    # 列順：1_to_2 → ... のFeature/SHAP交互順
    col_order = []
    for ratio in ratio_order:
        col_order.append(f"{ratio}_Feature")
        col_order.append(f"{ratio}_正規化SHAP影響度")
    col_order = ["構成", "Top"] + [col for col in col_order if col in pivoted.columns]
    pivoted = pivoted[col_order]

    # 書き出し処理（シート作成）
    worksheet = workbook.add_worksheet(model)

    # 色定義：Featureごとに固定色
    feature_colors = {}
    color_list = [
        "#D9E1F2", "#FCE4D6", "#E2EFDA", "#FFF2CC", "#F8CBAD",
        "#E4DFEC", "#DDEBF7", "#FBE4D5", "#DAE8FC", "#D5F5E3",
        "#F4CCCC", "#D9D2E9", "#C9DAF8", "#B6D7A8", "#FFE599"
    ]
    color_index = 0

    # ヘッダー
    for col_idx, col_name in enumerate(pivoted.columns):
        worksheet.write(0, col_idx, col_name)

    # データ行
    for row_idx, row in pivoted.iterrows():
        for col_idx, col_name in enumerate(pivoted.columns):
            value = row[col_name]
            if "Feature" in col_name and pd.notnull(value):
                if value not in feature_colors:
                    feature_colors[value] = workbook.add_format({"bg_color": color_list[color_index % len(color_list)]})
                    color_index += 1
                fmt = feature_colors[value]
                worksheet.write(row_idx + 1, col_idx, value, fmt)
            else:
                worksheet.write(row_idx + 1, col_idx, value)

workbook.close()
print(f"✅ 構成順を固定し、モデル別シートに出力しました: {output_path}")


✅ 構成順を固定し、モデル別シートに出力しました: SHAP影響度_モデル別シート_構成順付き.xlsx


In [12]:
import pandas as pd

# ① 各構成×モデル×データ比ごとに正規化SHAP値を計算
normalized_all = []
for (structure, model, ratio), group in combined_df.groupby(["構成", "モデル", "データ比"]):
    shap_df = (
        group.groupby("Feature")["SHAP平均影響度"]
        .mean()
        .reset_index()
    )
    shap_df["正規化SHAP影響度"] = shap_df["SHAP平均影響度"] / shap_df["SHAP平均影響度"].sum()
    shap_df["構成"] = structure
    shap_df["モデル"] = model
    shap_df["データ比"] = ratio
    normalized_all.append(shap_df)

normalized_df = pd.concat(normalized_all, ignore_index=True)

# ② Featureごとに平均・出現回数・分散を集計
summary = (
    normalized_df.groupby("Feature")
    .agg(
        出現回数=("正規化SHAP影響度", "count"),
        SHAP平均=("正規化SHAP影響度", "mean"),
        SHAP分散=("正規化SHAP影響度", "var")
    )
    .reset_index()
)

# ③ 総合スコア = 平均 × 出現回数、安定性スコア = 平均 / 分散
summary["総合スコア"] = summary["SHAP平均"] * summary["出現回数"]
summary["安定性スコア"] = summary["SHAP平均"] / (summary["SHAP分散"] + 1e-6)

# ④ スコア順に並べてTOP20を抽出
top20_features = summary.sort_values("総合スコア", ascending=False).head(20)


print(top20_features.head(20))  # テキスト表示
top20_features.to_excel("SHAP_top20_features.xlsx", index=False)  # Excel保存


                               Feature  出現回数    SHAP平均    SHAP分散     総合スコア  \
14189                    貸借対照表_利益剰余金合計    84  0.040137  0.009936  3.371543   
15629                      貸借対照表_長期預け金    84  0.034406  0.023688  2.890076   
5433          キャッシュ・フロー計算書_税引前当期純利益・損失    84  0.023327  0.022415  1.959468   
26909                    連結貸借対照表_利益剰余金    84  0.020958  0.002127  1.760505   
9846                        損益計算書_経常利益    84  0.019089  0.001625  1.603467   
10717                    損益計算書_関係会社清算損    84  0.018128  0.012510  1.522715   
7765                      損益計算書_会員権評価損    84  0.015563  0.010718  1.307330   
4828                   キャッシュ・フロー計算書_小計    84  0.012599  0.002493  1.058298   
7049                     損益計算書_その他事業原価    84  0.012382  0.006663  1.040083   
6227                          原価明細書_小計    84  0.012222  0.006350  1.026668   
7505                   損益計算書_不動産事業等売上高    84  0.010344  0.004405  0.868880   
7503                  損益計算書_不動産事業等売上原価    84  0.010331  0.004408

In [11]:
top20_features

,Feature,出現回数,SHAP平均,SHAP分散,総合スコア,安定性スコア
14189,貸借対照表_利益剰余金合計,84,0.040137,0.009936,3.371543,4.039066
15629,貸借対照表_長期預け金,84,0.034406,0.023688,2.890076,1.452383
5433,キャッシュ・フロー計算書_税引前当期純利益・損失,84,0.023327,0.022415,1.959468,1.040627
26909,連結貸借対照表_利益剰余金,84,0.020958,0.002127,1.760505,9.846725
9846,損益計算書_経常利益,84,0.019089,0.001625,1.603467,11.742798
10717,損益計算書_関係会社清算損,84,0.018128,0.012510,1.522715,1.448955
7765,損益計算書_会員権評価損,84,0.015563,0.010718,1.307330,1.452015
4828,キャッシュ・フロー計算書_小計,84,0.012599,0.002493,1.058298,5.052490
7049,損益計算書_その他事業原価,84,0.012382,0.006663,1.040083,1.857979
6227,原価明細書_小計,84,0.012222,0.006350,1.026668,1.924574
